In [80]:
# Trade Reconciliation Between Internal and Clearer Data

#This notebook aims to reconcile the internal trade data with the clearer’s official monthly settlement data. We will:
#- Load both datasets
#- Decompose quarterly contracts
#- Aggregate monthly positions
#- Compare quantities and prices with clearer data
#- Identify mismatches

In [81]:
!pip install openpyxl


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [82]:
import pandas as pd

# Load datasets
internal = pd.read_excel("internal_trade_data.xlsx")
clearer = pd.read_excel("clearer_trade_data.xlsx")

# Display first few rows to understand structure
display(internal.head())
display(clearer.head())

,trade_id,contract_type,contract_period,quantity_mwh,price_eur_per_mwh,Unnamed: 5
0,1.0,Outright,2025-03,100.0,30.0,NaN
1,2.0,Outright,2025-04,200.0,31.0,NaN
2,3.0,Outright,2025-05,250.0,32.0,NaN
3,4.0,Outright Quarter,Q225,300.0,33.0,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN


,delivery_month,quantity_mwh,price_eur_per_mwh
0,2025-03,100,30.0
1,2025-04,500,32.0
2,2025-05,550,32.5
3,2025-06,300,33.0


In [83]:
### Understanding Data Structure

#- The internal dataset contains outright monthly contracts and a quarterly contract (Q225).
#- The clearer dataset provides monthly settled quantities and prices.
#- The goal is to decompose the quarterly contract into monthly legs and net with monthly outright trades.


In [84]:
# Extract quarterly contract Q225
q225 = internal[(internal['contract_type'] == 'Outright Quarter') & (internal['contract_period'] == 'Q225')]
monthly = internal[internal['contract_type'] == 'Outright'].copy()

# Rename for clarity
monthly.rename(columns={'contract_period': 'month'}, inplace=True)

# Decompose quarterly contract evenly into three months
if not q225.empty:
    q225_qty = q225.iloc[0]['quantity_mwh']
    q225_price = q225.iloc[0]['price_eur_per_mwh']
    q225_months = ['2025-04', '2025-05', '2025-06']
    
    q225_df = pd.DataFrame({
        'month': q225_months,
        'quantity_mwh': [q225_qty / 3] * 3,
        'price_eur_per_mwh': [q225_price] * 3
    })
else:
    q225_df = pd.DataFrame(columns=['month', 'quantity_mwh', 'price_eur_per_mwh'])

# Combine monthly outright and decomposed quarterly contracts
monthly_all = pd.concat([monthly[['month', 'quantity_mwh', 'price_eur_per_mwh']], q225_df], ignore_index=True)


In [85]:
### Calculate weighted average price per month

#For months with both monthly and quarterly trades, calculate total quantity and weighted average price.

In [86]:
def weighted_avg(group):
    total_qty = group['quantity_mwh'].sum()
    if total_qty == 0:
        return 0
    return (group['quantity_mwh'] * group['price_eur_per_mwh']).sum() / total_qty

internal_agg = monthly_all.groupby('month').apply(
    lambda g: pd.Series({
        'internal_qty': g['quantity_mwh'].sum(),
        'internal_price': weighted_avg(g)
    })
).reset_index()

/tmp/ipykernel_244123/2027994497.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  internal_agg = monthly_all.groupby('month').apply(


In [87]:
### Prepare clearer data

#Rename columns to match internal data for easier merging.

In [88]:
clearer.rename(columns={
    'delivery_month': 'month',
    'quantity_mwh': 'clearer_qty',
    'price_eur_per_mwh': 'clearer_price'
}, inplace=True)

In [89]:
### Merge internal and clearer data

#- Join on month
#- Compare quantities and prices
#- Flag mismatches

In [90]:
final = pd.merge(internal_agg, clearer, on='month', how='outer').fillna(0)

# Quantity difference and matching flag
final['qty_diff'] = final['internal_qty'] - final['clearer_qty']
final['qty_match'] = final['qty_diff'].abs() < 1e-6

# Price difference and matching flag
final['price_diff'] = final['internal_price'] - final['clearer_price']
final['price_match'] = final['price_diff'].abs() < 0.01

# Format for display
final_display = final[['month', 'internal_qty', 'clearer_qty', 'qty_match', 'qty_diff', 'internal_price', 'clearer_price', 'price_match', 'price_diff']]
final_display.round(2)

,month,internal_qty,clearer_qty,qty_match,qty_diff,internal_price,clearer_price,price_match,price_diff
0,2025-03,100.0,100,True,0.0,30.00,30.0,True,0.00
1,2025-04,300.0,500,False,-200.0,31.67,32.0,False,-0.33
2,2025-05,350.0,550,False,-200.0,32.29,32.5,False,-0.21
3,2025-06,100.0,300,False,-200.0,33.00,33.0,True,0.00


In [93]:
internal_agg.to_csv('result_trade_reconciliation.csv', index=False)
print("file CSV exported correctly.")

file CSV exported correctly.


In [91]:
### Conclusion

#- The reconciliation shows which months quantities and prices match.
#- Differences highlight potential mismatches to investigate.

In [92]:
# Trade Reconciliation - Resumen

#Este notebook realiza la reconciliación de contratos de energía:
#- Se extrae el contrato trimestral Q225 y se descompone en sus meses correspondientes.
#- Se combinan los contratos mensuales y los meses descompuestos.
#- Se calcula la cantidad total y el precio promedio ponderado por mes.
#- El resultado se exporta en un archivo CSV llamado `trade_reconciliation_output.csv`.

#El código está preparado para facilitar la futura integración con otros datasets.
